In [1]:
# Import general libraries
import numpy as np
import pandas as pd
import re
import pickle
import torch
import itertools
from brian2 import *
from sbi import utils, inference
import matplotlib.pyplot as plt
import os
from pathlib import Path
import getpass
import psutil
import torch
from copy import deepcopy
from dataclasses import fields


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\franc\miniconda3\envs\mapping_RI_env\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\franc\miniconda3\envs\mapping_RI_env\Lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "c:\Users\franc\miniconda3\envs\mapping_RI_env\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start
    self

In [2]:
# Import from other files in the project
from simulator import SimulationParameters, run_simulation
from analyzer import AnalyzesParams, analyze_data

In [ ]:
# PRIOR AND PRIOR SAMPLING PARAMETERS FOR THE SIMULATION BATCH
batch_name_original = "Example_small_simulation_batch"
add_to_same_batch_simulation_folder = True # True # if True, this will add the new simulations to the existing folder instead of automatically creating a new folder

fixed_parameters = {
    "output_plots": False, # True # False
    "pre_specify_random_seed": False,
    "duration": 100, # 100
    "nb_pools": 1, # 2
    "nb_motoneurons_per_pool": 30, # 100
    "nb_RCs_per_pool_pair": 10, # 1/3 the nb of motor neurons
    "full_pool_sim": False, # False, # Set to "True" to have the full range of motor neuron sizes (from min to max), but with submaximal simulations a lot of neurons will remain silent
    "min_soma_diameter": 50, #50,
    "max_soma_diameter": 100, # 100,
    "mean_soma_diameter": 60, #55,
    "sd_prct_soma_diameter": 5.0, #0,
    #       ## Excitatory input baseline is a free parameter, bounded by posterior samples
    "nb_of_common_inputs": 2,
    "frequency_range_to_set_input_power": [0, 5], # min and max, in Hz
    "excitatory_input_baseline": [45*1e3, 45*1e3], # 25*1e3 for good baseline value when no RI, given MN size # one value per pool # 40*1e3 for RI disynaptic connectivity = 1
    #       ## The common input frequency range (of both pools), is defined by the posterior samples
    # "max_frequency_range_of_first_common_input": [20, 5], # one value per pool
    "frequency_range_of_common_input": np.array([ 
        [[0.0, 5.0], [15.0, 35.0]], # frequency range for pool 0 inputs [[low, high] for input 0], [[low, high] for input 1]
        [[0.0, 5.0], [15.0, 35.0]] # frequency range for pool 1 inputs [[low, high] for input 0], [[low, high] for input 1]
    ]),
    # ^ Comment out if setting mid- and half-width frequency of common input in the free parameters later
    # Instead, these values should be assigned (they will be written into frequency_range_of_common_input):
    # "common_input_high_freq_middle_of_range_pool0": 15,
    # "common_input_high_freq_half_width_range_pool0" 35,
    # "common_input_high_freq_middle_of_range_pool1": 25,
    # "common_input_high_freq_half_width_range_pool1": 10,
    "disynaptic_inhib_self_connectivity_pool0": 1.0, # 1.0, # if 1, then, on average, each MN makes one inhibitory disynaptic connection with any of the other motor neuron
    "disynaptic_inhib_self_connectivity_pool1": 0.0,
    "disynaptic_inhib_connectivity_pool0_to_pool1": 0.0,
    "disynaptic_inhib_connectivity_pool1_to_pool0": 0.0,
    #
    "distribution_params":{'std': 0.4, 'std_is_prct': True, 'ratio_large_small': 1.0},
    # net std of the MN->MN connectivity is sqrt(2*std**2 + std**4) / sqrt(nb_RCs_per_pool_pair). So with std=0.4 and nb_RCs_per_pool_pair=10, the std of MN->MN connectivity is ~0.20
    "split_MN_RC_ratio": 0.5, # 0.3,
    "disynaptic_inhib_received_arbitrary_adjustment": 0, # 0.4, # Adds randomness to the overall RI received by each MN - otherwise because of the averaging effect of connecting to RCs and then to MNs, the between-MN variability is very low
    #       ## Common input parameters
    "common_input_std": np.array([[3.5*1e3, 0.0*1e3], # common_input_std for pool 0 [input 0, input 1]
                                [3.5*1e3, 0.0*1e3]]), # common_input_std for pool 1 [input 0, input 1]
    "common_input_std_as_amp_or_prct": ['nA', 'nA'],
    "independent_input_absolute_or_ratio": 'ratio', # 'absolute' or 'ratio'
    "low_pass_filter_of_MN_independent_input": 80, # in Hz
    "max_frequency_of_any_input": 80, # in Hz
    "independent_input_power": 3*np.sqrt(2), # ratio # 10.0*1e3, # absolute
    "Renshaw_to_MN_IPSP": 5.0, # 10.0*1e-2, # 5.0*(5*1e-2), # 5.0 when scale_initial_IPSP_to_be_same_integral_regardless_of_synaptic_tau=False # 5.0*(5*1e-2) when scale_initial_IPSP_to_be_same_integral_regardless_of_synaptic_tau=True (= if IPSP is total charge), # it is converted to microAmps later - and if scale_initial_IPSP_to_be_same_integral_regardless_of_synaptic_tau=True, further converted into Coulombs (desired total inhibitory charges) in the simulator script
    # ^ If scale_initial_IPSP_to_be_same_integral_regardless_of_synaptic_tau==True, Renshaw_to_MN_IPSP should be set much lower, typically with *(5*1e-2)
    "set_arbitrary_correlation_between_excitatory_inputs": True,
    #       ## between_pool_excitatory_input_correlation is a free parameter
    # "between_pool_excitatory_input_correlation": 0.0,
    "scale_initial_IPSP_to_be_same_integral_regardless_of_synaptic_tau": False, # If false, shorter synaptic time constant will result in less inhibition
    "synaptic_IPSP_membrane_or_user_defined_time_constant": "membrane", # "membrane", # 'user_defined' or 'membrane'
}
free_parameter_bounds_prior = {
    # Parameters to sample from a uniform prior distribution
    "disynaptic_inhib_self_connectivity_pool0": [0.0, 3.0],
    'std_of_second_common_input_pool0': [0.0, 7.0*1e3],
    # "common_input_high_freq_middle_of_range_pool0": [15, 30],
    # "excitatory_input_baseline_pool0": [25.0*1e3, 45*1e3] # [30.0*1e3, 50*1e3],
    # 'std_of_second_common_input_pool0': [0.0, 5.0*1e3],
    # 'synaptic_IPSP_decay_time_constant': [5.0, 100.0] # in ms
    # 'MN_RC_synpatic_delay': [0.5, 15] # in ms

    # "disynaptic_inhib_connectivity_pool0_to_pool1": [0.0, 2.0],
    # "disynaptic_inhib_connectivity_pool1_to_pool0": [0.0, 2.0],
    # "between_pool_excitatory_input_correlation": [0.0, 1.0], # From no correlation (inputs are independent) to full correlation (inputs are identical)
    # ^ Makes the correlation relative to the common input of pool 0, so that the frequency range of the input of pool1 may not be respected if some correlation is enforced
}
free_parameters_to_sample_from_posterior = {
    # Parameters to override from posterior samples
    # # # Only used when simulation pairs of motor pool, extracting the inferred posterior for single pools from previously-performed SBI
    # "common_input_high_freq_middle_of_range_pool0",
    # "common_input_high_freq_half_width_range_pool0",
    # "common_input_high_freq_middle_of_range_pool1",
    # "common_input_high_freq_half_width_range_pool1",
    # "excitatory_input_baseline_pool0", # some amount will be added to the posterior-sampled value (to compensate for between-muscles RI)
    # "excitatory_input_baseline_pool1", # some amount will be added to the posterior-sampled value (to compensate for between-muscles RI)
    # 'std_of_second_common_input_pool0',
    # 'std_of_second_common_input_pool1',
    # "disynaptic_inhib_self_connectivity_pool0",
    # "disynaptic_inhib_self_connectivity_pool1",
}
baseline_excit_input_to_add_to_posterior_sample = 0 # 20*1e3 # in nA # randomly add between 0 and baseline_excit_input_to_add_to_posterior_sample to compensate for the between-muscles RI

sample_with_exponential = { # Instead of being sampled uniformly, the parameyters (keys) in this dict will be sampled according to an exponential rule (lower values are more likely to be sampled), with the exponent being specified as the value associated with the dict key
    # 'synaptic_IPSP_decay_time_constant': 0.02,
    # "min_frequency_range_of_first_common_input": 0.02,
    # "min_frequency_range_of_second_common_input": 0.02,
}
sample_at_specified_values = {
    # "common_input_high_freq_middle_of_range_pool0": [25.0], # [10, 20, 30, 40, 50],
    # 'std_of_second_common_input_pool0': [5.0*1e3],
    # 'std_of_second_common_input_pool0': [0.0*1e3],
    # "common_input_high_freq_half_width_range_pool0": [10.0] # [10], # "common_input_high_freq_half_width_range_pool0": [5, 10, 20, 30]
    # "excitatory_input_baseline_pool0": [50.0*1e3], # High enough to keep motor units active even with strong RI
    # 'std_of_second_common_input_pool0': [0.0*1e3],
    # "disynaptic_inhib_self_connectivity_pool0": [0, 1, 2, 3],
    # "synaptic_IPSP_decay_time_constant": [5.0, 10.0, 20.0, 30.0]
}
# ^ Instead of generating random priors from a uniform distribution, this will sample the simulator at specific parameter values.
# If there are 3 parameters and each of them have 3 values to be sampled, the simulator will simulate every possible combinations, so 3*3*3=27

torch.manual_seed(42)
nb_of_sims_from_prior = 10 # number of simulations that will be run in total, sampling randomly from the prior each time
# ^ if there are elements in 'sample_at_specified_values', each possible combination of 'sample_at_specified_values' will be assigned equal number of simulations, and the total will be as close as possible to nb_of_sims_from_prior
sim_parallel_cpus = 16


In [4]:
# # # IF DESIRED, SET SOME PARAMETERS TO BE SAMPLED FROM A PREVIOUSLY-ESTIMATED POSTERIOR
use_posterior_as_priors = False # True
path_for_posterior_samples = "" # "C://Users//franc//Documents//GitHub//SBI_motor_neuron_behavior//$$$_Simulation_batch_single_muscle//Z_inference//inference_most_up_to_date//posterior_samples_each_subject_df.csv"
muscle_pair_posterior = {0: "SOL", 1: "GM"} # {0: "SOL", 1: "GM"} # {0: "SOL", 1: "GM"} # {0: "VM", 1: "VL"}
            # {0: "VL", 1: "VM"} # identical to {0: "VM", 1: "VL"} in the sampling later, as the parameters are set both ways anyways
            # {0: "SOL", 1: "GM"}
            # {0: "GM", 1: "SOL"}
intensity_posterior = 10 # 10 or 40
if use_posterior_as_priors:
    batch_name_original = f"{batch_name_original}{muscle_pair_posterior[0]}-{muscle_pair_posterior[1]}_intensity{intensity_posterior}"

In [5]:
# SAMPLE PRIOR (for this simulation batch) FROM POSTERIOR (of previous inference) AND CREATE A DATA FRAME

# # # Get posterior data frame (if desired)
_param_from_posterior_field_map = {
    "excitatory_input_baseline": "excitatory_input_baseline_",
    "disynpatic_inhib_connections_desired_MN_MN": "disynaptic_inhib_self_connectivity_",
    "common_input_high_freq_middle_of_range": "common_input_high_freq_middle_of_range_",
    "common_input_high_freq_half_width_range": "common_input_high_freq_half_width_range_",
    "common_input_std": "std_of_second_common_input_",
}

if use_posterior_as_priors:
    # 1) load & preprocess once
    posterior_df = (
        pd.read_csv(path_for_posterior_samples, index_col=0)
        # strip off “<->” from muscle_pair
        .assign(muscle = lambda df: df['muscle_pair'].str.split('<->').str[0])
    )

    # 2) filter by intensity
    df_sub = posterior_df[posterior_df['intensity'] == intensity_posterior]

    # 3) split out the two pools & keep only subjects in both
    pool0 = df_sub[df_sub['muscle']==muscle_pair_posterior[0]]
    pool1 = df_sub[df_sub['muscle']==muscle_pair_posterior[1]]

    common_subjects = np.intersect1d(pool0['subject'].unique(),
                                    pool1['subject'].unique())

    pool0 = pool0[pool0['subject'].isin(common_subjects)].reset_index()
    pool1 = pool1[pool1['subject'].isin(common_subjects)].reset_index()

    # 4) draw all your random‐sample indices at once
    n_rows = len(pool0)
    # (we assume pool0 and pool1 have the same length after the subject‐intersection)
    indices = np.random.randint(0, n_rows, size=nb_of_sims_from_prior)

    # 5) pull out those rows in one go
    s0 = pool0.iloc[indices].reset_index(drop=True)
    s1 = pool1.iloc[indices].reset_index(drop=True)

    # 6) assemble the final DataFrame in one shot
    data = {
        'subject'  : s0['subject'].values,   # scalar per row
        'intensity': s0['intensity'].values, # scalar per row
        'muscle'   : [np.array([m0, m1]) 
                    for m0, m1 in zip(s0['muscle'], s1['muscle'])]
    }

    for field_name, param_name in _param_from_posterior_field_map.items():
        v0 = s0[field_name].values
        v1 = s1[field_name].values

        if param_name.endswith('_'):
            # e.g. "disynaptic_inhib_self_connectivity_"
            # split into two real columns
            col0 = f"{param_name}pool0"
            col1 = f"{param_name}pool1"
            data[col0] = v0.tolist()
            data[col1] = v1.tolist()
        else:
            # pack into a single 2-element array column
            data[param_name] = [np.array([a, b]) 
                                for a, b in zip(v0, v1)]

    priors_from_posterior_df = pd.DataFrame(data, index=range(nb_of_sims_from_prior))
# nb of rows in priors_from_posterior_df = nb_of_sims_from_prior
else:
    priors_from_posterior_df = pd.DataFrame([]) # Leave as an empty data frame

In [6]:
# # # SAMPLE FROM THE UNIFORM-PRIOR FREE PARAMETERS AND CREATE A DATA FRAME
prior_from_uniform_distrib_df = {}

def sample_truncated_exponential(low, high, exponent, size):
    """
    Draw `size` samples from the density
      p(x) = exponent * exp(-exponent*(x-low))  / (1 - exp(-exponent*(high-low)))
    for x in [low,high].
    This makes smaller x more likely, with rate=lam.
    """
    u = np.random.rand(*size)  # uniform [0,1)
    Z = 1 - np.exp(-exponent*(high - low))   # normalization denominator
    return low - (1.0/exponent)*np.log(1 - u * Z)

for free_param_name, free_param_bounds in free_parameter_bounds_prior.items():
    # print(free_param_name)
    # print(free_param_bounds)
    # print(np.ndim(free_param_bounds))
    arr = np.asarray(free_param_bounds)
    if arr.ndim == 1:
        # simple [low, high]
        low, high = arr
        if free_param_name in sample_with_exponential.keys():
            exponent = sample_with_exponential[free_param_name]
            print(f"Sampling {free_param_name} from an exponential distribution (λ={exponent})")
            random_sample_from_prior = sample_truncated_exponential(low, high, exponent,
                      size=(nb_of_sims_from_prior,))
        else:
            random_sample_from_prior = np.random.uniform(low, high, size=nb_of_sims_from_prior)
    elif arr.ndim == 2:
        # vector bounds: shape (n_dims, 2)
        lows  = arr[:, 0]   # length = n_dims
        highs = arr[:, 1]
        # draws an array (nb_sims × n_dims)
        if free_param_name in sample_with_exponential.keys():
            exponent = sample_with_exponential[free_param_name]
            print(f"Sampling {free_param_name} from an exponential distribution (λ={exponent})")
            # one exponential draw per dimension per sample
            random_sample_from_prior = np.column_stack([
                sample_truncated_exponential(l, h, exponent, size=(nb_of_sims_from_prior,))
                for l, h in zip(lows, highs)
            ])
        else:
            random_sample_from_prior = np.random.uniform(lows, highs,
                                        size=(nb_of_sims_from_prior, arr.shape[0]))
    else:
        raise ValueError(f"Can’t handle bounds of dimension {arr.ndim} for '{free_param_name}'")
    
    prior_from_uniform_distrib_df[free_param_name] = list(random_sample_from_prior)

prior_from_uniform_distrib_df = pd.DataFrame(prior_from_uniform_distrib_df,
                        index=range(nb_of_sims_from_prior))

In [7]:
prior_samples_df = pd.concat([priors_from_posterior_df, prior_from_uniform_distrib_df], axis=1)
prior_samples_df

,disynaptic_inhib_self_connectivity_pool0,common_input_high_freq_middle_of_range_pool0
0,1.173123,25.698362
1,2.938910,18.098686
2,0.829456,25.454304
3,1.053371,15.000287
4,2.017923,15.896367
5,0.399770,25.419486
6,0.998662,18.569864
7,0.031324,28.088887
8,1.658368,28.932537
9,1.163232,18.525137


In [8]:
# Divide prior_sample_df into equal nb of rows for each combination of 'sample_at_specified_values', and assign new columns with names = keys of 'sample_at_specified_values' and values = value combinations of 'sample_at_specified_values'
# Returns an error if not at least one simulaton per combination
# Will return the max nb of simulations so that each combination 'sample_at_specified_values' has the same number of elements while making sure that total_nb_ofsims (rows in prior_samples_df) <= nb_of_sims_from_prior
def assign_specified_values(prior_samples_df: pd.DataFrame,
                            sample_at_specified_values: dict,
                            shuffle: bool = True,
                            random_state: int | None = None) -> pd.DataFrame:
    """
    Split prior_samples_df evenly across all combinations of `sample_at_specified_values`
    and add new columns (keys) filled with the corresponding combination values.

    - If `sample_at_specified_values` is empty, returns df unchanged.
    - Errors if there aren't enough rows to give ≥1 row per combination.
    - Uses the maximum even allocation: sims_per_combo = floor(N / n_combos).
    - Drops leftover rows so all combinations have equal counts.
    """
    # Handle empty dict gracefully
    if not sample_at_specified_values:
        print("[info] No specified values provided; returning dataframe unchanged.")
        return prior_samples_df

    # Build combinations (preserve key order)
    items = list(sample_at_specified_values.items())
    keys  = [k for k, _ in items]
    grids = [v for _, v in items]
    combos = list(itertools.product(*grids))
    n_combos = len(combos)
    N = len(prior_samples_df)

    print(f"[info] available rows = {N}")
    print(f"[info] number of combinations = {n_combos}")

    sims_per_combo = N // n_combos
    if sims_per_combo < 1:
        raise ValueError(
            f"Not enough simulations: need ≥ {n_combos} rows (one per combination), "
            f"but only have {N}."
        )

    total_used = sims_per_combo * n_combos
    discarded = N - total_used
    print(f"[info] sims per combination = {sims_per_combo}")
    print(f"[info] total rows used = {total_used}"
          + (f" (discarding {discarded} extra rows to keep groups equal)" if discarded else ""))

    # Optionally shuffle to avoid bias, then keep only the needed rows
    idx = np.arange(N)
    rng = np.random.default_rng(random_state)
    if shuffle:
        rng.shuffle(idx)
    idx_used = idx[:total_used]

    df = prior_samples_df.iloc[idx_used].copy()
    df.reset_index(drop=True, inplace=True)

    # Assign values in consecutive blocks per combination
    for combo_i, combo_vals in enumerate(combos):
        start = combo_i * sims_per_combo
        end   = start + sims_per_combo
        for k, v in zip(keys, combo_vals):
            df.loc[start:end-1, k] = v

    print("[done] assignment complete.")
    return df

prior_samples_df = assign_specified_values(prior_samples_df, sample_at_specified_values,
                                      shuffle=True)

# Quick sanity check
print("Final total rows:", len(prior_samples_df))
prior_samples_df


[info] No specified values provided; returning dataframe unchanged.
Final total rows: 10


,disynaptic_inhib_self_connectivity_pool0,common_input_high_freq_middle_of_range_pool0
0,1.173123,25.698362
1,2.938910,18.098686
2,0.829456,25.454304
3,1.053371,15.000287
4,2.017923,15.896367
5,0.399770,25.419486
6,0.998662,18.569864
7,0.031324,28.088887
8,1.658368,28.932537
9,1.163232,18.525137


In [9]:
# # # Checking which upper bound should be used for the excitatory baseline
# posterior_df_temp = posterior_df[posterior_df['muscle']=='VL']
# posterior_df_temp = posterior_df_temp[posterior_df_temp['intensity']==40]
# np.max(posterior_df_temp.index)

In [10]:
# 1) A mapping of input-names → functions that take (value, init_kwargs, default)
#    and mutate init_kwargs appropriately.
_special_setters = { # for parameters which should be set with their Brian2 units directly
    'Renshaw_to_MN_IPSP': lambda v, kw, default: kw.update({
         'Renshaw_to_MN_IPSP': v * 1e3 * nA}),
    'AHP_conductance_delta_after_spiking': lambda v, kw, default: kw.update({
         'AHP_conductance_delta_after_spiking': v * msiemens}),
    'tau_Renshaw': lambda v, kw, default: kw.update({
         'tau_Renshaw': v * ms}),
    'synaptic_IPSP_decay_time_constant': lambda v, kw, default: kw.update({
         'synaptic_IPSP_decay_time_constant': v * ms}),
    'MN_RC_synpatic_delay': lambda v, kw, default: kw.update({
         'MN_RC_synpatic_delay': v * ms}),
}

def all_values_filled(d): # helper function
    """
    Recursively check if all values in a nested dict are not None.
    """
    for v in d.values():
        if isinstance(v, dict):
            if not all_values_filled(v):  # recurse
                return False
        else:
            if v is None:
                return False
    return True

def build_simulation_parameters(fixed_parameters, free_priors, batch_name):
    default = SimulationParameters()
    init_kwargs = {}
    dataclass_fields = {f.name for f in fields(SimulationParameters)}

    # Combine them — free overrides fixed if same key
    all_priors = {**fixed_parameters, **free_priors}

    # (optional) prepare your disynaptic matrix once if you still want
    disyn = deepcopy(default.disynpatic_inhib_connections_desired_MN_MN)
    disyn_touched = False
    high_freq_input_frequency = {"pool0": {"middle_of_range": None, "half_width": None},
                                "pool1": {"middle_of_range": None, "half_width": None}}
    # ^ Carefull when controlling high_freq_input_frequency: if those values are not all assigned manually, default/fixed values will be kept!

    for name, val in all_priors.items():

        # ------------------------------------------------------------------------
        # 1) if there's a special setter for this key, run it:
        if name in _special_setters:
            _special_setters[name](val, init_kwargs, default)
            continue

        # ------------------------------------------------------------------------
        # 2) handle any of the “multi‐field” cases:
        # (mostly set by different variables, with "pool0" or "pool1" specified), so this part is not used most of the time
        if name == 'excitatory_input_baseline':
            init_kwargs['excitatory_input_baseline'] = list(val); continue
        if name == 'common_input_std':
            cis = deepcopy(default.common_input_std)
            for i, v in enumerate(val):
                cis[i] = v
            init_kwargs['common_input_std'] = cis
            continue

        # 3) handle parameters which do not have direct corresponding names in the simulator Dataclass
        # (mostly correspond to sepcific entries in multi-filed variables, setting values specific to a given pool or input)
        # ------------------------------------------------------------------------
        # Input frequency range
        if name.startswith(("common_input_high_freq_")):
            pool_idx = int(name[-1]) # Get the last string as the pool index
            if "middle_of_range" in name:
                high_freq_input_frequency[f"pool{pool_idx}"]["middle_of_range"] = val
            elif "half_width" in name:
                high_freq_input_frequency[f"pool{pool_idx}"]["half_width"] = val
            if (all_values_filled(high_freq_input_frequency)):
                for row_i in [0,1]:
                    mid = high_freq_input_frequency[f"pool{row_i}"]["middle_of_range"]
                    half = high_freq_input_frequency[f"pool{row_i}"]["half_width"]
                    init_kwargs['frequency_range_of_common_input'] = deepcopy(
                        init_kwargs.get(
                            'frequency_range_of_common_input',
                            default.frequency_range_of_common_input
                        )
                    )
                    init_kwargs['frequency_range_of_common_input'][row_i, 1] = [mid - half, mid + half]
            continue

        # ------------------------------------------------------------------------
        # Excitatory input baseline
        if name.startswith("excitatory_input_baseline_"):
            # determine which pool to write into
            pool = 0 if "0" in name else 1
            # start from whatever you’ve already set, or fallback to the default array
            excit_input_baseline_temp = deepcopy(
                init_kwargs.get(
                    "excitatory_input_baseline",
                    default.excitatory_input_baseline
                )
            )
            excit_input_baseline_temp[pool] = val + np.random.uniform(0, baseline_excit_input_to_add_to_posterior_sample)
            init_kwargs["excitatory_input_baseline"] = excit_input_baseline_temp
            continue

        # ------------------------------------------------------------------------
        # Common‐input STD of first/second input (for each pool) in one block
        if name.startswith("std_of_"):
            # determine which column (input 0 vs input 1) to write into
            input = 0 if "first" in name else 1
            pool = 0 if "pool0" in name else 1
            # start from whatever you’ve already set, or fallback to the default array
            cis = deepcopy(
                init_kwargs.get(
                    "common_input_std",
                    default.common_input_std
                )
            )
            cis[pool, input] = val
            init_kwargs["common_input_std"] = cis
            continue

        # ------------------------------------------------------------------------
        # disynaptic connectivity
        if name == 'disynaptic_inhib_self_connectivity_pool0':
            disyn[0,0] = val; disyn_touched = True; continue
        if name == 'disynaptic_inhib_self_connectivity_pool1':
            disyn[1,1] = val; disyn_touched = True; continue
        if name == 'disynaptic_inhib_connectivity_pool0_to_pool1':
            disyn[0,1] = val; disyn_touched = True; continue
        if name == 'disynaptic_inhib_connectivity_pool1_to_pool0':
            disyn[1,0] = val; disyn_touched = True; continue

        # ------------------------------------------------------------------------
        # 3) If it matches a real dataclass field, assign it directly:
        if name in dataclass_fields:
            init_kwargs[name] = val
            continue

        raise KeyError(f"Unrecognized parameter '{name}'")

    # 4) Write back the disynaptic matrix once, if we tweaked it
    if disyn_touched:
        init_kwargs['disynpatic_inhib_connections_desired_MN_MN'] = disyn

    # 5) Always override the folder name
    init_kwargs['output_folder_name'] = batch_name

    # 6) Build the object (this runs __post_init__ just once, with all your overrides in place)
    return SimulationParameters(**init_kwargs)


In [11]:
# # # CREATE FOLDER FOR THE SIMULATION BATCH
if add_to_same_batch_simulation_folder: # Do not create a new folder - just add the new simulations to the existing one with the same name
    batch_name = batch_name_original
    os.makedirs(batch_name, exist_ok=True)
    print(f"Using batch folder: {batch_name}")
else:
    # figure out the next batch index - if creating a new folder
    # find all subdirectories matching "{batch_name_original}_{digits}"
    pattern = re.compile(rf"^{re.escape(batch_name_original)}_(\d+)$")
    batch_dirs = [
        d for d in os.listdir('.')
        if os.path.isdir(d) and pattern.match(d)
    ]

    if batch_dirs:
        # extract the numeric suffixes
        existing_idxs = [int(pattern.match(d).group(1)) for d in batch_dirs]
        batch_idx = max(existing_idxs) + 1
    else:
        batch_idx = 0

    batch_name = f"{batch_name_original}_{batch_idx}"
    os.makedirs(batch_name, exist_ok=True)

    print(f"Created batch folder: {batch_name}")

Using batch folder: Example_simulation_batch


In [12]:
prior_samples_df

,disynaptic_inhib_self_connectivity_pool0,common_input_high_freq_middle_of_range_pool0
0,1.173123,25.698362
1,2.938910,18.098686
2,0.829456,25.454304
3,1.053371,15.000287
4,2.017923,15.896367
5,0.399770,25.419486
6,0.998662,18.569864
7,0.031324,28.088887
8,1.658368,28.932537
9,1.163232,18.525137


In [13]:
params_prior_list = []
for i in range(len(prior_samples_df)):
    free_row = prior_samples_df.iloc[i].to_dict()
    # remove elements which are not parameters - namely the subject/intensity/muscle the parameters sampled from posteriors are extracted from
    if "subject" in free_row:
        free_row.pop("subject")
    if "intensity" in free_row:
        free_row.pop("intensity")
    if "muscle" in free_row:
        free_row.pop("muscle")
    sim_param = build_simulation_parameters(fixed_parameters, free_row, batch_name)
    params_prior_list.append(sim_param)

In [14]:
# Import for parallelization
from joblib import Parallel, delayed
from brian2 import prefs, device
import logging

# # # Function to make sure to terminate any Python process that runs in the background (this can happen when the kernel crashes during the parallelized computations)
def kill_other_python_processes():
    me = os.getpid()
    user = getpass.getuser()
    for proc in psutil.process_iter(['pid', 'name', 'username']):
        try:
            # only consider Python executables run by this user
            if proc.info['username'] != user:
                continue
            name = proc.info['name'].lower()
            # match python, pythonw, python3, etc
            if name.startswith('python'):
                pid = proc.info['pid']
                if pid != me:
                    proc.kill()   # or proc.terminate()
                    print(f"{name} process terminated")
        except (psutil.NoSuchProcess, psutil.AccessDenied):
            pass
if __name__ == '__main__':
    kill_other_python_processes()
    # now safe to start joblib Parallel(...)

# --- Helper to wrap a single simulation and then reset Brian2 ---
def _run_and_reset(params):
    # params must be a SimulationParameters instance
    out = run_simulation(params)
    # after each run, reset Brian2’s magic network so the next worker starts fresh
    device.reinit()     # clears all Brian2 objects
    device.activate()   # re–activate the default runtime device
    return out

# # # PARALLEL SIMULATIONS
def parallel_simulate(params_prior_list, n_jobs=8):
    """
    params_prior_list : list of SimulationParameters
    n_jobs      :       number of parallel workers
    """
    # Note: `prefer="processes"` is the default for `n_jobs>1`
    sim_outputs = Parallel(n_jobs=n_jobs)(
        delayed(_run_and_reset)(p) for p in params_prior_list)
    return sim_outputs

In [15]:
# # # SAVE ALL THE PRIORS AND SAMPLED PRIOR PARAMETERS FOR THE GIVEN BATCH
# save all the prior info into a pickle
to_save = {
    'fixed_parameters':         fixed_parameters,
    'free_parameter_bounds_prior':    free_parameter_bounds_prior,
    'free_parameters_to_sample_from_posterior': free_parameters_to_sample_from_posterior,
    'torch_seed':               torch.initial_seed(),  # or whatever seed you used
    'num_samples':              nb_of_sims_from_prior,
    'prior_samples_df':         prior_samples_df,
    'params_prior_list':        params_prior_list,
    'use_posterior_as_priors':  use_posterior_as_priors,
}
if use_posterior_as_priors:
    to_save['priors_from_posterior_df'] = priors_from_posterior_df
    to_save['muscle_pair_posterior'] = muscle_pair_posterior
    to_save['intensity_posterior'] = intensity_posterior
    to_save['path_for_posterior_samples'] = path_for_posterior_samples
    
# pkl_path = os.path.join(batch_name, f"{batch_name}_priors.pkl")
# with open(pkl_path, 'wb') as f:
#     pickle.dump(to_save, f)

# print(f"✅ Saved priors to {pkl_path}")

# Prepare the base filename
base = f"{batch_name}_priors"
if "\\" in base:
    base = base.split("\\")[-1]

# Find existing files of the form "{base}_batch<idx>.pkl"
pattern = re.compile(rf"^{re.escape(base)}_batch(\d+)\.pkl$")
existing_idxs = []
for fn in os.listdir(batch_name):
    m = pattern.match(fn)
    if m:
        existing_idxs.append(int(m.group(1)))

# Choose the next index
if existing_idxs:
    next_idx = max(existing_idxs) + 1
else:
    next_idx = 0

# Build the new filename
filename = f"{base}_batch{next_idx}.pkl"
pkl_path = os.path.join(batch_name, filename)

# Save
with open(pkl_path, 'wb') as f:
    pickle.dump(to_save, f)

print(f"✅ Saved priors to {pkl_path}")

# Tag each SimulationParameters with the batch folder
for p in params_prior_list:
    p.output_folder_name = batch_name


✅ Saved priors to Example_simulation_batch\Example_simulation_batch_priors_batch0.pkl


In [16]:
# Capture progres (in .log file) directly in the notebook cell
from threading import Thread, Event
import time
from datetime import datetime
import re

_tail_thread = None
_tail_stop = threading.Event()

def start_tail(logfile="simulations_progress_log.log", poll_interval=1.0):
    """
    Spawn a thread that prints only the log‐lines whose timestamp
    is ≥ the moment you called start_tail(), and strips off everything
    before the log‐level (INFO:, WARNING:, ERROR:, etc.).
    """
    global _tail_thread, _tail_stop

    # make sure the file exists (touch it)
    open(logfile, "a").close()

    # remember "now" and clear any previous stop flag
    start_dt = datetime.now()
    _tail_stop.clear()

    def _tail_loop():
        level_re = re.compile(r'\b(?:DEBUG|INFO|WARNING|ERROR|CRITICAL)\b:\s*')
        with open(logfile, "r", encoding="utf-8") as f:
            # seek to end: we only want new lines
            f.seek(0, 2)
            while not _tail_stop.is_set():
                line = f.readline()
                if not line:
                    time.sleep(poll_interval)
                    continue

                # try to parse timestamp at the very start
                try:
                    ts_str = " ".join(line.split(" ")[:2]).rstrip(",")
                    ts = datetime.strptime(ts_str, "%Y-%m-%d %H:%M:%S,%f")
                except Exception:
                    ts = start_dt  # force print for non‐timestamped lines

                if ts >= start_dt:
                    # strip off everything before the level marker
                    m = level_re.search(line)
                    if m:
                        print(line[m.start():], end="")
                    else:
                        print(line, end="")

    # fire up the thread (only one at a time)
    if _tail_thread is None or not _tail_thread.is_alive():
        _tail_thread = threading.Thread(target=_tail_loop, daemon=True)
        _tail_thread.start()
    else:
        print("Tail already running; call `end_tail()` first if you want to restart.")

def stop_tail():
    """Stop the background tail thread."""
    _tail_stop.set()
    if _tail_thread:
        _tail_thread.join()


In [17]:
# Run parallel simulations
start_tail()
simulation_output_files = parallel_simulate(params_prior_list, n_jobs=sim_parallel_cpus)
time.sleep(1)  # give it a moment to print the last lines
stop_tail()

INFO: Initializing simulation 0...
INFO: Initializing simulation 1...
INFO: Initializing simulation 2...
INFO: Initializing simulation 3...
INFO: Initializing simulation 4...
INFO: Initializing simulation 5...
INFO: Initializing simulation 6...
INFO: Initializing simulation 7...
INFO: Initializing simulation 8...
INFO: Initializing simulation 9...
INFO: building '_cython_magic_06d9fffc0ff069e9d817eaef05f8a0a3' extension
INFO: "C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.43.34808\bin\HostX86\x64\cl.exe" /c /nologo /O2 /W3 /GL /DNDEBUG /MD -Ic:\Users\franc\miniconda3\envs\mapping_RI_env\Library\include -Ic:\Users\franc\miniconda3\envs\mapping_RI_env\Lib\site-packages\brian2\synapses -Ic:\Users\franc\miniconda3\envs\mapping_RI_env\include -Ic:\Users\franc\miniconda3\envs\mapping_RI_env\Include "-IC:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.43.34808\include" "-IC:\Program Files (x86)\Microsoft Visual Studio\2022\BuildT